In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Reading the "credits.csv" file
credits_df = pd.read_csv("credits.csv")

# Reading the "movies.csv" file
movies_df = pd.read_csv("movies.csv")

In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
movies_df = movies_df.merge(credits_df,on='title')

In [ ]:
movies_df.shape

In [ ]:
movies_df.info()

In [ ]:
movies_df=movies_df[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.dropna(inplace=True)

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.iloc[0].genres

In [ ]:
import ast

In [ ]:
def Convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies_df['genres']=movies_df['genres'].apply(Convert)
movies_df['keywords']=movies_df['keywords'].apply(Convert)

In [ ]:
def Convert3 (obj) :
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter +=1
    else :
      break
      return L

In [ ]:
movies_df['cast']=movies_df['cast'].apply(Convert3)

In [ ]:
def fetch_director (obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      return L

In [ ]:
movies_df['crew']=movies_df['crew'].apply(fetch_director)

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

def replace_spaces(lst):
    if lst is not None:
        return [i.replace(" ", "") for i in lst]
    return lst

movies_df['cast'] = movies_df['cast'].apply(replace_spaces)
movies_df['crew'] = movies_df['crew'].apply(replace_spaces)

In [ ]:
def create_tag(row):
    tag_elements = []

    if isinstance(row['overview'], str):
        tag_elements.append(row['overview'])

    if isinstance(row['genres'], list):
        tag_elements.extend(row['genres'])

    if isinstance(row['keywords'], list):
        tag_elements.extend(row['keywords'])

    if isinstance(row['cast'], list):
        tag_elements.extend(row['cast'])

    if isinstance(row['crew'], list):
        tag_elements.extend(row['crew'])

    return ' '.join(tag_elements)

movies_df['tag'] = movies_df.apply(create_tag, axis=1)

In [ ]:
new_df=movies_df[['movie_id','title','tag']]

In [ ]:
new_df


In [ ]:
new_df['tag']=new_df['tag'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer (max_features= 5000,stop_words= 'english')

In [ ]:
tag_matrix = cv.fit_transform(new_df['tag']).toarray()
tag_matrix.shape

In [ ]:
vectors = tag_matrix

In [ ]:
vectors[0]

In [ ]:
feature_names = cv.get_feature_names_out()
print("Number of features:", len(feature_names))

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tag']=new_df['tag'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
similarity[0].shape

In [ ]:
sorted(list(enumerate(similarity[0])),reverse = True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend (movie):
  movie_index = new_df[new_df['title']==movie].index[0]
  distance = similarity[movie_index]
  movies_list=sorted(list(enumerate(distance)),reverse = True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Ant-Man')

In [ ]:
from flask_cors import CORS
from flask import Flask, request, jsonify

app = Flask(__name__)
CORS(app)

@app.route('/recommend', methods=['POST'])
def get_recommendations():
    try:
        data = request.get_json()
        movie = data.get('movie')

        if movie is None:
            return jsonify({'error': 'Movie parameter is missing'}), 400

        # Call your recommendation function
        recommendations = recommend(movie)

        return jsonify({'recommendations': recommendations})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)
